In [ ]:
# запускаем в colab или локально?
try:
    from google.colab import drive
    colab = True
except ImportError:
    colab = False

print(f"colab: {colab}")

In [ ]:
# установка необходимых пакетов в colab
if colab:
    ! pip install rootutils -q
    ! pip install livelossplot -q

In [ ]:
# монтирование google диска и установка
# рабочей директории в `computer-vision`

import os
from rootutils import setup_root

if colab:
    drive.mount("/content/drive", force_remount=True)
    os.chdir("drive/MyDrive/computer-vision")
    root = setup_root(".", indicator="homeworks", pythonpath=True)
else:
    root = setup_root(".", indicator="homeworks", pythonpath=True)

os.chdir(root)
print(f"working directory: {os.getcwd()}")

In [ ]:
# создание директории для данных

from pathlib import Path

if colab:
    DATA_DIR = Path("/content/data")
else:
    DATA_DIR = root / "data"

DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")

In [ ]:
# настройка matplotlib

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format="retina"

plt.style.use("seaborn-v0_8-notebook")

## **Основы PyTorch. Обучение нейронных сетей.**

### **1. Фреймворки для обучения нейронных сетей**

**Основные:**

- [PyTorch + Torchvision](https://pytorch.org)

- [TensorFlow](https://www.tensorflow.org)
- [JAX](https://docs.jax.dev)

**Более высокоуровневые:**
- [Keras](https://keras.io) - в качестве бэкенда можно использовать PyTorch, JAX или TensorFlow

- [Flax](https://flax.readthedocs.io) - основан на JAX
- [PyTorch Lightning](https://lightning.ai/) - основан на PyTorch

### **2. PyTorch тензоры**

- Тензоры похожи на массивы в numpy.

- В отличае от numpy массивов, вычисления с тензорами можно делать на GPU.

- Тензоры позволяют вычислять производные.

Учебник по PyTorch: [60-min blitz.](https://docs.pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)

#### **2.1 Настройка ускорителя**

Ускоритель можно изменить в Google Colab, выбрав среду выполнения.

Проверим, доступен ли GPU и выберем ускоритель для вычислений:

In [ ]:
import torch

# проверим, доступен ли GPU
gpu_ok = torch.cuda.is_available()

print(f"\nGPU is available: {gpu_ok}")

# общий способ выбрать ускоритель для вычислений
device = torch.device("cuda:0" if gpu_ok else "cpu")

print(f"\nDevice being used for operations: {device}")

#### **2.2 Воспроизводимость вычислений на GPU**

Для воспроизводимости результатов вычислений необходимо установить seed:

In [ ]:
from src.utils import set_seed

set_seed(seed=42)

#### **2.3 Копирование c CPU на GPU и обратно**

Скопируем тензор из памяти CPU в память GPU:

In [ ]:
x = torch.zeros(2, 3)

x = x.to(device)

print("x", x)

Скопируем обратно с GPU на CPU:

In [ ]:
x = x.to("cpu")

print("x", x)

#### **2.4 Вычисления на CPU / GPU**

Измерим время умножения тензоров большого размера на CPU:

In [ ]:
import time

# случайный тензор размера 5000 x 5000 на CPU
x = torch.rand(5000, 5000)

# время умножения x * x на CPU
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()

cpu_time = end_time - start_time
print(f"CPU time: {cpu_time:6.5f}s")

Сравним со временем умножения тензоров на GPU:

In [ ]:
# должен быть доступен GPU
if gpu_ok:

    # копирование тензора с CPU на GPU
    x = x.to(device)

    # warmup GPU
    _ = torch.matmul(x, x)

    # вычисления на GPU асинхронные,
    # для измерения времени нужна синхронизация вычислений
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()
    _ = torch.matmul(x, x)
    end.record()

    # дождаться, когда все вычисления на GPU закончатся
    torch.cuda.synchronize()
    gpu_time = 0.001 * start.elapsed_time(end)

    print(f"GPU time: {gpu_time:6.5f}s")
    print(f"CPU/GPU time: {int(cpu_time / gpu_time)}")

#### **2.5 Tensor to Numpy and Numpy to Tensor**

Тензоры можно конвертировать в numpy массивы, и массивы обратно в тензоры.

Чтобы конвертировать массив в тензор, мы можем использовать функцию `torch.from_numpy`:

In [ ]:
import numpy as np

np_arr = np.array([[1, 2], [3, 4]])

tensor = torch.from_numpy(np_arr)

print("Numpy array:\n", np_arr)
print("\nPyTorch tensor:\n", tensor)

Для того чтобы преобразовать тензор обратно в массив, используем функцию `.numpy()`:

In [ ]:
tensor = torch.arange(4) 

np_arr = tensor.numpy()

print("PyTorch tensor:", tensor)
print("Numpy array:", np_arr)

Массивы и тензоры на CPU имеют общую память.

Изменение одного из них приводит к изменению другого.

#### **2.6 Вычисление производных с помощью алгоритма обратного распространения ошибки**

Для функции:
$$
    y = \frac{1}{N}\sum_{n=1}^N
    \left[
        (x_n+2)^2 + 3
    \right]
$$

вычислим производные:
$$
    \frac{\partial y}{\partial x_n}\qquad
    n=1,2,\ldots,N
$$  
аналитически и численно.

Вычислительный граф для $y$:
$$
    \mathbf{x} = (x_1\,,x_2\,,\ldots\,,x_N)
$$
$$
    a_n = x_n + 2
$$
$$
    b_n = a^2_n
$$
$$
    c_n = b_n + 3
$$
$$
    y = \frac{1}{N}\sum_{n=1}^N c_n
$$
<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/main/figures/01/computation_graph.png" width=200px/>
    </figure>
</center>


Аналитическое вычисление производных:

$$
    \frac{\partial y}{\partial x_n} =
    \frac{\partial y}{\partial c_n} \cdot
    \frac{\partial c_n}{\partial b_n} \cdot
    \frac{\partial b_n}{\partial a_n} \cdot
    \frac{\partial a_n}{\partial x_n}
$$
$$
    \frac{\partial y}{\partial x_n} =
    \frac{1}{N} \cdot
    1 \cdot
    2a_n \cdot
    1 =
    \frac{2}{N}\cdot a_n =
    \frac{2}{N}\cdot (x_n + 2)
$$

Например, если 
$$
    N = 3\,,\qquad
    \mathbf{x} = (0\,,1\,,2)
$$
тогда:
$$
    \frac{\partial y}{\partial x_n} = \frac{2}{3}(x_n + 2)
    = 4/3,2,8/3
$$

Вычислим эти производные числено с помощью алгоритма обратного распространения ошибки (backpropagation algorithm).

Зададим `x` как параметр по которому можно вычислять производные:

In [ ]:
N = 3

x = torch.arange(N, dtype=torch.float32, requires_grad=True)

print(f"x.requires_grad = {x.requires_grad}")
print(f"x = {x}")

Вычислительный граф для `y`:

In [ ]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()

print(f"y = {y}")

Backpropagation algorithm:

In [ ]:
# `retain_graph=True` - чтобы можно вычислять производные несколько раз
# производные будут складываться (накапливаться)
y.backward(retain_graph=True)

print(x.grad)

In [ ]:
# чтобы производные не накапливались
# их надо перед вычислениями обнулить
x.grad = torch.zeros_like(x.grad)

y.backward(retain_graph=True)

print(x.grad)

### **3. Нейронные сети**

- Нейронная сеть - это функция, зависящая от входных данных $\mathbf{x}$ и параметров (весов) $\mathbf{w}$:
$$
    \mathbf{y} = \mathbf{f}[\mathbf{x}\,, \mathbf{w}]
$$

- Вычислительный граф для нейронной сети как правило разбивается на слои.

#### **3.1 Пример: фильтр Габора**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/main/figures/01/gabor.jpg" width=600px/>
    </figure>
</center>

У **фильтра Габора:**
$$
    y = \sin(w_1 + 0.06\cdot w_2\cdot x)\cdot
    \exp\left[-\frac{(w_1 + 0.06\cdot w_2\cdot x)^2}{32}\right]
$$
два параметра:
$$
    \mathbf{w} = [w_1, w_2]
$$

#### **3.2 Нейронные сети в PyTorch**

В PyTorch, для задания нейронных сетей служит класс `torch.nn.Module`:

In [ ]:
from torch import nn

class GaborNet(nn.Module):
    def __init__(self, w1=0, w2=0):
        super().__init__()

        self.w1 = nn.Parameter(w1 * torch.ones(1), requires_grad=True)
        self.w2 = nn.Parameter(w2 * torch.ones(1), requires_grad=True)

    def forward(self, x):
        h = self.w1 + 0.06 * self.w2 * x

        a1 = torch.sin(h)
        a2 = torch.exp(-h ** 2 / 32)

        y = a1 * a2

        return y

Возьмем
$$
    w_1 = 3.0\,,
    \qquad
    w_2 = 15.0
$$

Создадим модель нейронной сети и выведем информацию о модели:

In [ ]:
w1 = 3.0
w2 = 15.0

# создадим модель нейронной сети
model = GaborNet(w1, w2)

# Выведем параметры модели
for name, p in model.named_parameters():
    print(f"Parameter {name} = {p.item()}")

Сохраним и загрузим модель:

In [ ]:
# сохранение модели
torch.save(model.state_dict(), DATA_DIR / "gabor.pth")

# новая модель с нулевыми параметрами
model = GaborNet()

# загрузка модели
state_dict = torch.load(DATA_DIR / "gabor.pth", weights_only=True)
model.load_state_dict(state_dict, strict=True)

# выведем параметры модели после загрузки
for name, p in model.named_parameters():
    print(f"Parameter {name} = {p.item()}")

Построим график
$$
    y = f[x, \mathbf{w}]\,,
    \qquad
    x\in[-15, 15]
$$

In [ ]:
N = 200
x = torch.linspace(-15, 15, N)

# отключить автоматическое дифференцирование и вычисление градиента
with torch.inference_mode():
    # вызывается функция model.forward(x)
    y = model(x)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(x, y, s=3, c="blue")
plt.xlabel("x")
plt.ylabel("y");

### **4. Задача регрессии для фильтра Габора**

Сгенерируем данные для обучения, валидации и тестирования модели Габора:
$$
    \textbf{train data} = \left\{x_i\,,y_i\right\}^N_{i=1}
$$

$$
    \textbf{val data} = \left\{x_i\,,y_i\right\}^M_{i=1}
$$

$$
    \textbf{test data} = \left\{x_i\,,y_i\right\}^K_{i=1}
$$

Данные сгенерируем для значений истинных (ground truth) параметров:
$$
    w_1 = 3.0\,,
    \qquad
    w_2 = 15.0
$$
добавив к $y$ нормальный шум:
$$
    y + \varepsilon\,,
    \qquad
    \varepsilon \sim N\left(0\,,\sigma^2\right)
$$

Обучим модель Габора предсказывать $y$ по $x$:
$$
    f[x, \mathbf{w}] = \widehat{y}
    \quad\approx\quad
    y
$$

In [ ]:
gt_w1 = 3.0
gt_w2 = 15.0

model = GaborNet(gt_w1, gt_w2)

with torch.inference_mode():

    # данные для обучения
    N = 300
    sigma = 0.1

    x_train = torch.linspace(-15, 15, N)
    y_train = model(x_train) + sigma * torch.randn(N)

    # данные для валидации
    M = 200
    sigma = 0.2

    x_val = torch.linspace(-15, 15, M)
    y_val = model(x_val) + sigma * torch.randn(M)

    # данные для тестирования
    K = 100
    sigma = 0.3

    x_test = torch.linspace(-15, 15, K)
    y_test = model(x_test) + sigma * torch.randn(K)

Построим полученные данные:

In [ ]:
plt.figure(figsize=(12, 3))

# plot training data
plt.subplot(131)
plt.scatter(x_train, y_train, s=4, c="blue")
plt.title("Training Data")

# plot validation data
plt.subplot(132)
plt.scatter(x_val, y_val, s=4, c="green")
plt.title("Validation Data")

# plot testing data
plt.subplot(133)
plt.scatter(x_test, y_test, s=4, c="red")
plt.title("Testing Data")

plt.tight_layout()

#### **4.1 Обучение нейронных сетей с помощью метода стохастического градиентного спуска**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/main/figures/01/train.png" width=600px/>
    </figure>
</center>

В методе **стохастического градиентного спуска (SGD)**:

- Обучение происходит по одному случайно выбранному сэмплу данных $\{\mathbf{x}_i,\mathbf{y}_i\}$.

- На каждом шаге обучения производные от функции потерь вычисляются методом обратного распространения ошибки.

- Параметры модели обновляются оптимизатором.

- Эпоха - единичный проход через весь обучающий набор данных.

В методе **Batch SGD**, на каждом шаге обучения выбирается **батч** - случайное подмножество обучающих данных:
$$
    \mathbf{batch} =
    \left\{\mathbf{x}_i\,,\mathbf{y}_i\right\}\,,
    \qquad
    i\in{I}_t
    \subset\left\{1,2,3,\ldots,N\right\}
    \quad\text{- индексы батча на шаге } t
$$
    
В методе **градиентного спуска (GD)** батч содержит все данные для обучения:
$$
    {I}_t = \left\{1,2,3,\ldots,N\right\}
$$

#### **4.2 Формирование батчей**

Будем формировать бачи c помощью объектов классов `Dataset` и `DataLoader`:

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GaborDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y


# dataset генерирует данные по одному сэмплу (x_i, y_i)
train_dataset = GaborDataset(x_train, y_train)

# loader генерирует батчи данных
train_loader = DataLoader(
    train_dataset,
    batch_size = 10,  # размер батча
    shuffle = True,   # перемешивать данные в конце каждой эпохе
    drop_last = True, # не использовать последний батч в эпохе,
                      # если его размер меньше batch_size
    num_workers = 0   # не создавать дополнительные процессы для генерации данных
)

# loader данных для валидации
val_dataset = GaborDataset(x_val, y_val)

val_loader = DataLoader(
    val_dataset,
    batch_size = len(val_dataset), # батч содержит все данные для валидации
    shuffle = False,
    drop_last = False,
    num_workers = 0
)

Протестируем загрузчики:

In [ ]:
train_batch = next(iter(train_loader))

x, y = train_batch

print(f"x = \n{x}")
print(f"y = \n{y}")

In [ ]:
val_batch = next(iter(val_loader))

x, y = val_batch

print(f"x.shape: {x.shape}")
print(f"y.shape: {y.shape}")

#### **4.3 Функция потерь**

Для задачи регрессии в качестве функции потерь возьмем **среднеквадратическую ошибку (MSE)**:
$$
    L_t =
    \frac{1}{|{I}_t|}
    \sum_{i\in{I}_t}(y_i - \widehat{y}_i)^2
$$

In [ ]:
from torch import nn

loss_fn = nn.MSELoss(reduction="mean")

x, y = next(iter(train_loader))

y_hat = model(x)

loss_value = loss_fn(y_hat, y)

print(f"Значение функции потерь: {loss_value}")

#### **4.4 Оптимизатор**

Оптимизацию будем выполнять с помощью алгоритма градиентного спуска:

$$
    \mathbf{w}_{t+1} =
    \mathbf{w}_t -
    \lambda\cdot\frac{\partial L_t}{\partial\mathbf{w}_t}
$$

In [ ]:
# начальные значения параметров модели
w1 = 2.0
w2 = 14.0

model = GaborNet(w1, w2)

optimizer = torch.optim.SGD(
    model.parameters(), # параметры модели
    lr=0.01             # темп обучения (learning rate)
)

# вычислим loss для батча
x, y = next(iter(train_loader))
y_hat = model(x)
loss_value = loss_fn(y_hat, y)

# выведем параметры модели перед оптимизацией
for name, p in model.named_parameters():
    print(f"Parameter before optimization {name} = {p.item()}")

# обнулим производные, чтобы не было их накопления
optimizer.zero_grad()

# вычисление производных методом обратного распространения ошибки
loss_value.backward()

# обновление параметров модели
optimizer.step()

# выведем параметры модели после оптимизации
for name, p in model.named_parameters():
    print(f"Parameter after optimization {name} = {p.item()}")

#### **4.5 Обучение и валидация модели**

Соберем все вместе и напишем полный цикл обучения и валидации модели
для задачи регрессии фильтра Габора.

Для оценки качества обучения модели будем также использовать среднеквадратичную ошибку (MSE)
на валидационных данных:
$$
    \mathbf{MSE} =
    \frac{1}{M}
    \sum^M_{i=1}(y_i - \widehat{y}_i)^2
$$


In [ ]:
from dataclasses import dataclass
from tqdm import tqdm
from livelossplot import PlotLosses

@dataclass
class Config:
    seed = 0xABCD
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_workers = 0
    # гиперпараметры - то, что влияет на качество предсказания обученной модели
    epochs = 200
    learning_rate = 0.1
    batch_size = 100
    w1 = 2
    w2 = 14

cfg = Config()

# для воспроизводимости результатов обучения
set_seed(cfg.seed)

# загрузчик данных для обучения
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg.batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=cfg.num_workers
)

# загрузчик данных для валидации
val_loader = DataLoader(
    val_dataset,
    batch_size=cfg.batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=cfg.num_workers
)

# модель с начальными значениями параметров
model = GaborNet(cfg.w1, cfg.w2)

# копируем модель на ускоритель
model = model.to(cfg.device)

optimizer = torch.optim.SGD(model.parameters(), lr=cfg.learning_rate)
loss_fn = nn.MSELoss(reduction="mean")

# для построения графиков во время обучения
plot = PlotLosses(figsize=(12, 3))

# лучшее значение метрики качества
best_metric = float("inf")

for epoch in tqdm(range(cfg.epochs)):
    # ЭПОХА ОБУЧЕНИЯ МОДЕЛИ

    # модель переведем в режим обучения
    model.train()

    # значения loss функции на батчах одной эпохи
    batch_loss = []

    # перебираем случайные батчи из обучающих данных
    for x, y in train_loader:
        # скопируем батч на ускоритель
        x = x.to(cfg.device)
        y = y.to(cfg.device)

        # предсказание модели
        y_hat = model(x)

        # вычисление loss функции по предсказанию `y_hat` и реальному `y`
        loss = loss_fn(y_hat, y.float())

        # обнулим все производные перед их вычислениями
        # чтобы не было gradient accumulation
        optimizer.zero_grad()

        # вычисление производных loss функция по параметрам
        # c помощью метода обратного распространения ошибки (backpropagation)
        loss.backward()

        # обновление параметров модели
        optimizer.step()

        # отсоединим loss от вычислительного графа и скопируем на CPU
        loss = loss.detach().cpu()

        batch_loss.append(loss)

    # среднее значение функции потерь в текущей эпохе обучения
    # на данных для обучения
    train_loss = torch.tensor(batch_loss).mean()

    # ЭПОХА ВАЛИДАЦИИ МОДЕЛИ

    # модель переведем в режим валидации
    model.eval()

    # значения loss функции на батчах одной эпохи
    batch_loss = []

    # перебираем батчи из валидационных данных
    for x, y in val_loader:
        x = x.to(cfg.device)
        y = y.to(cfg.device)

        # этот контекст рекомендуют использовать для ускорения вычислений при валидации
        with torch.no_grad():
            y_hat = model(x)

        loss = loss_fn(y_hat, y.float())

        # копируем на CPU, отсоединять от графа не надо
        loss = loss.cpu()

        batch_loss.append(loss)

    # среднее значение функции потерь в текущей эпохе обучения
    # на данных для валидации
    val_loss = torch.tensor(batch_loss).mean()

    # сохраняем модель, если на валидации метрика улучшилась
    if val_loss < best_metric:
        best_metric = val_loss
        torch.save(model.state_dict(), DATA_DIR / "gabor.pth")

    # построение графиков во время обучения
    plot.update({
        "loss": train_loss,
        "val_loss": val_loss
    })

    if epoch % 10 == 0:
        plot.send()

#### **4.6 Тестирование модели**

Загрузим веса обученной модели для тестирования:

In [ ]:
state_dict = torch.load(DATA_DIR / "gabor.pth", weights_only=True)

model = GaborNet()
model.load_state_dict(state_dict, strict=True)

# выведем параметры модели после обучения
for name, p in model.named_parameters():
    print(f"Parameter {name} = {p.item()}")

# истинное значение параметров
print(f"Ground truth w1 = {gt_w1}")
print(f"Ground truth w2 = {gt_w2}")


Протестируем модель на тестовых данных на CPU:

In [ ]:
# создадим загрузчик тестовых данных
test_dataset = GaborDataset(x_test, y_test)

test_loader = DataLoader(
    test_dataset,
    batch_size=10,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

# ТЕСТИРОВАНИЕ МОДЕЛИ
model.eval()
test_loss = []

y_pred = torch.empty(0)

for x, y in test_loader:
    with torch.inference_mode():
        y_hat = model(x)

    y_pred = torch.concatenate((y_pred, y_hat))

    loss = loss_fn(y_hat, y.float())
    test_loss.append(loss)

# среднее значение loss на тестовых данных
test_loss = torch.tensor(test_loss).mean()

print(f"Test Loss: {test_loss}")

plt.figure(figsize=(8, 4))
plt.scatter(x_test, y_test, s=20, c="blue")
plt.scatter(x_test, y_pred, s=20, c="red")
plt.xlabel("x")
plt.ylabel("y");

### **5. Бинарная классификация XOR датасета**

#### **5.1 XOR dataset**

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/main/figures/01/xor_dataset.png" width=400px/>
    </figure>
</center>

Сформируем данные для обучения и тестирования:

In [ ]:
import torch
from torch.utils.data import Dataset

class XORDataset(Dataset):
    def __init__(self, size, sigma=0.2):
        super().__init__()
        self.size = size
        self.sigma = sigma
        self.generate_continuous_xor()

    def generate_continuous_xor(self):
        points = torch.randint(low=0, high=2, size=(self.size, 2), dtype=torch.float32)
        labels = (points.sum(dim=1) == 1).long()
        points += self.sigma * torch.randn(points.shape)

        self.points = points
        self.labels = labels

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        x = self.points[index]
        y = self.labels[index]

        return x, y

In [ ]:
train_dataset = XORDataset(size=2500)

val_dataset = XORDataset(size=500)

test_dataset = XORDataset(size=200)

print("Size of train dataset:", len(train_dataset))
print("Train data point 0:", train_dataset[0])

Построим точки тестового датасета:

In [ ]:
def visualize_samples(points, labels):
    class_0 = points[labels == 0]
    class_1 = points[labels == 1]

    plt.figure(figsize=(4,4))
    plt.scatter(class_0[:, 0], class_0[:, 1], edgecolor="#333", label="Class 0")
    plt.scatter(class_1[:, 0], class_1[:, 1], edgecolor="#333", label="Class 1")
    plt.title("Dataset samples")
    plt.ylabel(r"$x_2$")
    plt.xlabel(r"$x_1$")
    plt.legend()

visualize_samples(test_dataset.points, test_dataset.labels)

#### **5.2 Модель классификатора**

В качестве классификатора возьмем неглубокую нейронную сеть:

<center>
    <figure>
        <img src="https://media.githubusercontent.com/media/alextanch/computer-vision/refs/heads/main/figures/01/small_net.png" width=300px/>
    </figure>
</center>

**Вычислительный граф сети**.

- вычисление преактивации
$$
    \begin{bmatrix}
        z_1 \\ z_2 \\ z_3 \\ z_4
    \end{bmatrix}
    =
    \begin{bmatrix}
        \omega_{11} & \omega_{12} \\
        \omega_{21} & \omega_{22} \\
        \omega_{31} & \omega_{32} \\
        \omega_{41} & \omega_{42}
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
        x_1 \\ x_2
    \end{bmatrix}
    +
    \begin{bmatrix}
        \beta_{1} \\ \beta_{2} \\ \beta_{3} \\ \beta_{4}
    \end{bmatrix}
$$

- вычисление скрытого слоя
$$
    \begin{bmatrix}
        h_1 \\ h_2 \\ h_3 \\ h_4
    \end{bmatrix}
    =
    \begin{bmatrix}
        a[z_1] \\ a[z_2] \\ a[z_3] \\ a[z_4]
    \end{bmatrix}
$$

где $a[z]$ - функция активации гиперболический тангенс:
$$
    a[z] = \tanh[z] =
    \frac{\exp{(z)}-\exp{(-z)}}{\exp{(z)}+\exp{(-z)}}
$$

- вычисление выходного слоя
$$
    y =
    \begin{bmatrix}
        \theta_1 & \theta_2 & \theta_3 & \theta_4
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
        h_1 \\ h_2 \\ h_3 \\ h_4
    \end{bmatrix}
    + \gamma
$$

Параметры сети - это все греческие буквы. Число параметров равно 17.

In [ ]:
from torch import nn

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(in_features=2, out_features=4)
        self.act = nn.Tanh()
        self.linear2 = nn.Linear(4, 1)

    def forward(self, x):
        z = self.linear1(x)
        h = self.act(z)
        y = self.linear2(h)
        return y

model = Classifier()

for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

#### **5.3 Функция потерь для задачи бинарной классификации**

**Binary Cross Entropy (BCE)**

$$
    \mathbf{BCE} =
    -\frac{1}{|{I}_t|}
    \sum_{i\in{I}_t}\left[y_i\ln p_i - (1-y_i)\ln(1-p_i)\right]
$$
где $y_i =\mathrm{label}_i$ - истинное значение метки класса, $p_i$ - предсказанная сетью вероятность принадлежности классу:
$$
        p_i = \mathrm{sigmoid}[\widehat{y}_i] =
        \frac{1}{1 + \exp(-\widehat{y}_i)}
$$
    
В PyTorch есть два способа вычислить **BCE**:

- с помощью `nn.BCEWithLogitsLoss()`. В этом случае, вероятности $p_i$ вычисляются в самой функции потерь.

- с помощью `nn.BCELoss()`. В этом случае, вероятности $p_i$ нужно вычислять в последнем слое нейронной сети.


In [ ]:
from torch import nn

loss_fn = nn.BCEWithLogitsLoss()

#### **5.4 Метрика качества для задачи бинарной классификации**

В качестве метрики качества возьмем точность (accuracy):

$$
    \mathbf{acc} =
    \frac{\#\text{correct predictions}}{\#\text{all predictions}}
$$